In [3]:
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier

from main import preprocess, best_params
from merge_df import table_to_dataframe
from tokenization import tokenize

Функция для кодирования разметки в числовые классы. **NB**: классы `neutral` и `speech` объединяются (чтобы было больше данных)

In [7]:
def encode_labeling(label):
    if label == 'negative':
        label_id = -1
    elif label == 'neutral':
        label_id = 0
    elif label == 'positive':
        label_id = 1
    elif label == 'speech':
        label_id = 0

    return label_id

Загружем данные

In [4]:
filepath = '../Data/data.xlsx'
raw_df = table_to_dataframe(filepath)
raw_df.dropna(axis=0, how='any', inplace=True)
raw_df.drop(columns=['index'], inplace=True)

Обрабатываем и проверяем, что не осталось пустых ячеек

In [8]:
data = raw_df[raw_df['разметка'] != 'skip'].copy()
data['tokenized'] = data['текст'].apply(tokenize)
data['processed'] = [preprocess(tok, params=best_params) for tok in data['tokenized']]

data.fillna('none',  inplace=True)
assert data.isnull().values.any() == False

Векторизуем и сохраняем векторайзер в файл

In [14]:
corpus = data['processed']
vectorizer = TfidfVectorizer(analyzer=str.split)
X = vectorizer.fit_transform(corpus)

with open('../web/vectorizer.pickle', 'wb') as f:
    pickle.dump(vectorizer, f)

Кодируем разметку, обучаем классификатор и сохраняем в файл

In [15]:
SEED = 67

y = data['разметка'].apply(encode_labeling)

clf = SGDClassifier(random_state=SEED)
clf.fit(X, y)

with open('../web/classifier.pickle', 'wb') as f:
    pickle.dump(clf, f)